In [15]:
import scipy.io as sio
import os
import sys
import numpy as np
import pandas as pd
import time
import pickle
import torch
from tqdm import tqdm
import math
import scipy

In [140]:
import scipy.io as sio
import os
import sys
import numpy as np
import pandas as pd
import time
import pickle
import torch
from tqdm import tqdm
import math
import scipy

def data_1Dto2D(data, Y=9, X=9):
    data_2D = np.zeros([Y, X])
    data_2D[0] = (0,        0,          0,          data[0],    0,          data[16],   0,          0,          0       )
    data_2D[1] = (0,        0,          0,          data[1],    0,          data[17],   0,          0,          0       )
    data_2D[2] = (data[3],  0,          data[2],    0,          data[18],   0,          data[19],   0,          data[20])
    data_2D[3] = (0,        data[4],    0,          data[5],    0,          data[22],   0,          data[21],   0       )
    data_2D[4] = (data[7],  0,          data[6],    0,          data[23],   0,          data[24],   0,          data[25])
    data_2D[5] = (0,        data[8],    0,          data[9],    0,          data[27],   0,          data[26],   0       )
    data_2D[6] = (data[11], 0,          data[10],   0,          data[15],   0,          data[28],   0,          data[29])
    data_2D[7] = (0,        0,          0,          data[12],   0,          data[30],   0,          0,          0       )
    data_2D[8] = (0,        0,          0,          data[13],   data[14],   data[31],   0,          0,          0       )
    return data_2D


def norm_dataset(dataset_1D):
    norm_dataset_1D = np.zeros([dataset_1D.shape[0], 32])
    for i in range(dataset_1D.shape[0]):
        norm_dataset_1D[i] = feature_normalize(dataset_1D[i])
    return norm_dataset_1D


def feature_normalize(data):
    mean = data[data.nonzero()].mean()
    sigma = data[data. nonzero ()].std()
    data_normalized = data
    data_normalized[data_normalized.nonzero()] = (data_normalized[data_normalized.nonzero()] - mean)/sigma
    return data_normalized


def dataset_1Dto2D(dataset_1D):
    dataset_2D = np.zeros([dataset_1D.shape[0],9,9])
    for i in range(dataset_1D.shape[0]):
        dataset_2D[i] = data_1Dto2D(dataset_1D[i])
    return dataset_2D


def norm_dataset_1Dto2D(dataset_1D):
    norm_dataset_2D = np.zeros([dataset_1D.shape[0], 9, 9])
    for i in range(dataset_1D.shape[0]):
        norm_dataset_2D[i] = feature_normalize(data_1Dto2D(dataset_1D[i]))
    return norm_dataset_2D


def windows(data, size):
    start = 0
    while ((start+size) < data.shape[0]):
        yield int(start), int(start + size)
        start += size

        
def segment_signal_without_transition(data,label,label_index,window_size):
    for (start, end) in windows(data, window_size):
        if((len(data[start:end]) == window_size)):
            if(start == 0):
                segments = data[start:end]
                segments = np.vstack([segments, data[start:end]])

#                 labels = np.array(label[label_index])
#                 labels = np.append(labels, np.array(label[label_index]))
            else:
                segments = np.vstack([segments, data[start:end]])
#                 labels = np.append(labels, np.array(label[label_index])) 
    return segments, np.array(label[label_index])


def apply_mixup(dataset_file,window_size,label,basemean):
    print("Processing",dataset_file,"..........")
    data_file_in = sio.loadmat(dataset_file)
    print("original EEG data size:", data_file_in["data"].shape)
    data_in = data_file_in["data"].transpose(0,2,1)
    print("transpose EEG data size:", data_in.shape)
    #0 valence, 1 arousal, 2 dominance, 3 liking
    if label=="arousal":
        label=1
    elif label=="valence":
        label=0
    label_in= data_file_in["labels"][:,label]
    label_inter = np.empty([0]) 
    data_inter_E = np.empty([0, 60*window_size, 9, 9])
    data_inter_C = np.empty([0, 60*window_size, 32])
    DE_FEATURE = np.empty([0, 60, 9, 9]) 
    trials = data_in.shape[0]


    for trial in range(0,trials):
        if basemean=="yes":
            base_signal = (data_in[trial,0:128,0:32]+data_in[trial,128:256,0:32]+data_in[trial,256:384,0:32])/3
            if trial == 0: print("40번 반복")
            if trial == 0: print("base signal data size", base_signal.shape)
        else:
            base_signal = np.zeros([128,32])
        data_E = data_in[trial,384:8064,0:32]
        # data_C = data_in[trial,384:8064,0:32]
        
        DE = np.empty([60, 32]) 

        for i in range(0,60):
            for k in range(0,32):
                DE[i, k] = scipy.stats.differential_entropy(data_E[i*128:(i+1)*128, k]-base_signal[:, k])
        label_index = trial

        if trial == 0: print("DE before norm data size:", DE.shape)
        DE = norm_dataset(DE)
        if trial == 0: print("DE after norm data size:", DE.shape)
        DE, label = segment_signal_without_transition(DE, label_in, label_index, 1)
        # data_C = norm_dataset(data_C)
        # data_C, label = segment_signal_without_transition(data_C, label_in, label_index, window_size)

        if trial == 0: print("DE segment signal without transition data size:", DE.shape)
        DE = dataset_1Dto2D(DE)
        if trial == 0: print("DE 1D to 2D data size:", DE.shape)
        DE = DE.reshape (1, DE.shape[0], 9, 9)
        if trial == 0: print("DE reshape data size:", DE.shape)

        # data_C = data_C.reshape(1, data_C.shape[0], 32)

        DE_FEATURE  = np.vstack([DE_FEATURE, DE])
        # data_inter_C  = np.vstack([data_inter_C, data_C])
        # label_inter = np.append(label_inter, label)
    
    print("Total DE_FEATURE data size:", DE_FEATURE.shape)
    # print("EEG-ContextNet data size:", data_inter_C.shape)
    # print("total label size:", label_inter.shape)
    

    return DE_FEATURE ,data_inter_C, label_inter

### DE Feature Extraction

#### Training dataset

##### save pkl

In [125]:
begin = time.time()
print("time begin:",time.localtime())

dataset_dir = "/workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab"
window_size = 128
output_dir = "/workspace/G_Project/DEAP_Dataset/Feature_Extraction/"
label_class = "arousal"     # arousal/valence
basemean = "yes"     # yes/no (using baseline signals or not)

record_list = [task for task in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir,task))]
output_EEG_dir = output_dir+"DE_feature/"+f"basemean_{basemean}"+"/"
if os.path.isdir(output_EEG_dir)==False:
    os.makedirs(output_EEG_dir)

    
for record in record_list:
    file = os.path.join(dataset_dir,record)
    data_inter_E,data_inter_C,label_inter = apply_mixup(file, window_size,label_class,basemean)
    trials = data_inter_E.shape[0]

    for trial in range(0,trials):
        output_data_cnn = output_EEG_dir+record.split(".")[0]+f"_trial{str(trial+1).zfill(2)}"+"_win_"+str(window_size)+".pkl"

        with open(output_data_cnn, "wb") as fp:
            pickle.dump(data_inter_E[trial],fp, protocol=4)
    end = time.time()
    print("end time:",time.localtime())
    print("time consuming:",(end-begin))

time begin: time.struct_time(tm_year=2023, tm_mon=8, tm_mday=16, tm_hour=14, tm_min=53, tm_sec=0, tm_wday=2, tm_yday=228, tm_isdst=0)
Processing /workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab/s24.mat ..........
original EEG data size: (40, 40, 8064)
transpose EEG data size: (40, 8064, 40)
40번 반복
base signal data size (128, 32)
DE before norm data size: (60, 32)
DE after norm data size: (60, 32)
DE segment signal without transition data size: (60, 32)
DE 1D to 2D data size: (60, 9, 9)
DE reshape data size: (1, 60, 9, 9)
Total DE_FEATURE data size: (40, 60, 9, 9)
end time: time.struct_time(tm_year=2023, tm_mon=8, tm_mday=16, tm_hour=14, tm_min=53, tm_sec=7, tm_wday=2, tm_yday=228, tm_isdst=0)
time consuming: 7.059483766555786
Processing /workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab/s23.mat ..........
original EEG data size: (40, 40, 8064)
transpose EEG data size: (40, 8064, 40)
40번 반복
base signal data size (128, 32)
DE before norm data size: (60, 32)
DE after nor

In [132]:
begin = time.time()
print("time begin:",time.localtime())

dataset_dir = "/workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab"
window_size = 128
output_dir = "/workspace/G_Project/DEAP_Dataset/Feature_Extraction/"
label_class = "arousal"     # arousal/valence
basemean = "no"     # yes/no (using baseline signals or not)

record_list = [task for task in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir,task))]
output_EEG_dir = output_dir+"DE_feature/"+f"basemean_{basemean}"+"/"
if os.path.isdir(output_EEG_dir)==False:
    os.makedirs(output_EEG_dir)

    
for record in record_list:
    file = os.path.join(dataset_dir,record)
    data_inter_E,data_inter_C,label_inter = apply_mixup(file, window_size,label_class,basemean)
    trials = data_inter_E.shape[0]

    for trial in range(0,trials):
        output_data_cnn = output_EEG_dir+record.split(".")[0]+f"_trial{str(trial+1).zfill(2)}"+"_win_"+str(window_size)+".pkl"

        with open(output_data_cnn, "wb") as fp:
            pickle.dump(data_inter_E[trial],fp, protocol=4)
    end = time.time()
    print("end time:",time.localtime())
    print("time consuming:",(end-begin))

time begin: time.struct_time(tm_year=2023, tm_mon=8, tm_mday=16, tm_hour=15, tm_min=0, tm_sec=42, tm_wday=2, tm_yday=228, tm_isdst=0)
Processing /workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab/s24.mat ..........
original EEG data size: (40, 40, 8064)
transpose EEG data size: (40, 8064, 40)
DE before norm data size: (60, 32)
DE after norm data size: (60, 32)
DE segment signal without transition data size: (60, 32)
DE 1D to 2D data size: (60, 9, 9)
DE reshape data size: (1, 60, 9, 9)
Total DE_FEATURE data size: (40, 60, 9, 9)
end time: time.struct_time(tm_year=2023, tm_mon=8, tm_mday=16, tm_hour=15, tm_min=0, tm_sec=49, tm_wday=2, tm_yday=228, tm_isdst=0)
time consuming: 7.0601725578308105
Processing /workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab/s23.mat ..........
original EEG data size: (40, 40, 8064)
transpose EEG data size: (40, 8064, 40)
DE before norm data size: (60, 32)
DE after norm data size: (60, 32)
DE segment signal without transition data size: (60, 32

##### save .pt

In [133]:
dataset_dir = "/workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab"
window_size = 128
output_dir = "/workspace/G_Project/DEAP_Dataset/Feature_Extraction/"
basemean = "yes"     # yes/no (using baseline signals or not)

record_list = sorted([task for task in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir,task))])
output_EEG_dir = output_dir+"DE_feature/"+f"basemean_{basemean}_pt"+"/"
if os.path.isdir(output_EEG_dir)==False:
    os.makedirs(output_EEG_dir)

    
for record in tqdm(record_list):
    file = os.path.join(dataset_dir,record)
    data_inter_E,data_inter_C,label_inter = apply_mixup(file, window_size,label_class,basemean)
    trials = data_inter_E.shape[0]

    for trial in range(0,trials):
        output_data_cnn_path = output_EEG_dir+record.split(".")[0]+f"_trial{str(trial+1).zfill(2)}"+"_win_"+str(window_size)+".pt"
        output_data_cnn_tensor = torch.tensor(data_inter_E[trial])

        torch.save(output_data_cnn_tensor, output_data_cnn_path)

  0%|          | 0/32 [00:00<?, ?it/s]

Processing /workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab/s01.mat ..........
original EEG data size: (40, 40, 8064)
transpose EEG data size: (40, 8064, 40)
40번 반복
base signal data size (128, 32)
DE before norm data size: (60, 32)
DE after norm data size: (60, 32)
DE segment signal without transition data size: (60, 32)
DE 1D to 2D data size: (60, 9, 9)
DE reshape data size: (1, 60, 9, 9)


  3%|▎         | 1/32 [00:07<03:37,  7.01s/it]

Total DE_FEATURE data size: (40, 60, 9, 9)
Processing /workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab/s02.mat ..........
original EEG data size: (40, 40, 8064)
transpose EEG data size: (40, 8064, 40)
40번 반복
base signal data size (128, 32)
DE before norm data size: (60, 32)
DE after norm data size: (60, 32)
DE segment signal without transition data size: (60, 32)
DE 1D to 2D data size: (60, 9, 9)
DE reshape data size: (1, 60, 9, 9)


  6%|▋         | 2/32 [00:14<03:30,  7.00s/it]

...

Total DE_FEATURE data size: (40, 60, 9, 9)
Processing /workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab/s31.mat ..........
original EEG data size: (40, 40, 8064)
transpose EEG data size: (40, 8064, 40)
40번 반복
base signal data size (128, 32)
DE before norm data size: (60, 32)
DE after norm data size: (60, 32)
DE segment signal without transition data size: (60, 32)
DE 1D to 2D data size: (60, 9, 9)
DE reshape data size: (1, 60, 9, 9)


 97%|█████████▋| 31/32 [03:37<00:07,  7.00s/it]

Total DE_FEATURE data size: (40, 60, 9, 9)
Processing /workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab/s32.mat ..........
original EEG data size: (40, 40, 8064)
transpose EEG data size: (40, 8064, 40)
40번 반복
base signal data size (128, 32)
DE before norm data size: (60, 32)
DE after norm data size: (60, 32)
DE segment signal without transition data size: (60, 32)
DE 1D to 2D data size: (60, 9, 9)
DE reshape data size: (1, 60, 9, 9)


100%|██████████| 32/32 [03:44<00:00,  7.01s/it]

Total DE_FEATURE data size: (40, 60, 9, 9)


In [134]:
dataset_dir = "/workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab"
window_size = 128
output_dir = "/workspace/G_Project/DEAP_Dataset/Feature_Extraction/"
basemean = "no"     # yes/no (using baseline signals or not)

record_list = sorted([task for task in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir,task))])
output_EEG_dir = output_dir+"DE_feature/"+f"basemean_{basemean}_pt"+"/"
if os.path.isdir(output_EEG_dir)==False:
    os.makedirs(output_EEG_dir)

    
for record in tqdm(record_list):
    file = os.path.join(dataset_dir,record)
    data_inter_E,data_inter_C,label_inter = apply_mixup(file, window_size,label_class,basemean)
    trials = data_inter_E.shape[0]

    for trial in range(0,trials):
        output_data_cnn_path = output_EEG_dir+record.split(".")[0]+f"_trial{str(trial+1).zfill(2)}"+"_win_"+str(window_size)+".pt"
        output_data_cnn_tensor = torch.tensor(data_inter_E[trial])

        torch.save(output_data_cnn_tensor, output_data_cnn_path)

  0%|          | 0/32 [00:00<?, ?it/s]

Processing /workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab/s01.mat ..........
original EEG data size: (40, 40, 8064)
transpose EEG data size: (40, 8064, 40)
DE before norm data size: (60, 32)
DE after norm data size: (60, 32)
DE segment signal without transition data size: (60, 32)
DE 1D to 2D data size: (60, 9, 9)
DE reshape data size: (1, 60, 9, 9)


  3%|▎         | 1/32 [00:07<03:38,  7.05s/it]

Total DE_FEATURE data size: (40, 60, 9, 9)
Processing /workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab/s02.mat ..........
original EEG data size: (40, 40, 8064)
transpose EEG data size: (40, 8064, 40)
DE before norm data size: (60, 32)
DE after norm data size: (60, 32)
DE segment signal without transition data size: (60, 32)
DE 1D to 2D data size: (60, 9, 9)
DE reshape data size: (1, 60, 9, 9)


  6%|▋         | 2/32 [00:14<03:29,  7.00s/it]

...

Total DE_FEATURE data size: (40, 60, 9, 9)
Processing /workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab/s31.mat ..........
original EEG data size: (40, 40, 8064)
transpose EEG data size: (40, 8064, 40)
DE before norm data size: (60, 32)
DE after norm data size: (60, 32)
DE segment signal without transition data size: (60, 32)
DE 1D to 2D data size: (60, 9, 9)
DE reshape data size: (1, 60, 9, 9)


 97%|█████████▋| 31/32 [03:36<00:06,  6.99s/it]

Total DE_FEATURE data size: (40, 60, 9, 9)
Processing /workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab/s32.mat ..........
original EEG data size: (40, 40, 8064)
transpose EEG data size: (40, 8064, 40)
DE before norm data size: (60, 32)
DE after norm data size: (60, 32)
DE segment signal without transition data size: (60, 32)
DE 1D to 2D data size: (60, 9, 9)
DE reshape data size: (1, 60, 9, 9)


100%|██████████| 32/32 [03:43<00:00,  6.99s/it]

Total DE_FEATURE data size: (40, 60, 9, 9)


##### save .pt

In [138]:
dataset_dir = "/workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab"
window_size = 128
output_dir = "/workspace/G_Project/DEAP_Dataset/Feature_Extraction/"
basemean = "yes"     # yes/no (using baseline signals or not)

record_list = sorted([task for task in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir,task))])
output_EEG_dir = output_dir+"DE_feature/test/"+f"basemean_{basemean}_pt"+"/"
if os.path.isdir(output_EEG_dir)==False:
    os.makedirs(output_EEG_dir)

    
for record in tqdm(record_list):
    file = os.path.join(dataset_dir,record)
    data_inter_E,data_inter_C,label_inter = apply_mixup_test(file, window_size,label_class,basemean)
    trials = data_inter_E.shape[0]

    for trial in range(0,trials):
        output_data_cnn_path = output_EEG_dir+record.split(".")[0]+f"_trial{str(trial+1).zfill(2)}"+"_win_"+str(window_size)+".pt"
        output_data_cnn_tensor = torch.tensor(data_inter_E[trial])

        torch.save(output_data_cnn_tensor, output_data_cnn_path)

  0%|          | 0/32 [00:00<?, ?it/s]

Processing /workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab/s01.mat ..........


/opt/conda/lib/python3.10/site-packages/scipy/stats/_entropy.py:378: RuntimeWarning: divide by zero encountered in log
  logs = np.log(n * differences / (ci * m))
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:232: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/tmp/ipykernel_8945/1446177570.py:33: RuntimeWarning: invalid value encountered in subtract
  data_normalized[data_normalized.nonzero()] = (data_normalized[data_normalized.nonzero()] - mean)/sigma
  0%|          | 0/32 [00:00<?, ?it/s]

original EEG data size: (40, 40, 8064)
transpose EEG data size: (40, 8064, 40)
40번 반복
base signal data size (128, 8)
DE before norm data size: (60, 8)


IndexError: index 60 is out of bounds for axis 0 with size 60

In [ ]:
dataset_dir = "/workspace/G_Project/DEAP_Dataset/data_preprocessed_matlab"
window_size = 128
output_dir = "/workspace/G_Project/DEAP_Dataset/Feature_Extraction/"
basemean = "no"     # yes/no (using baseline signals or not)

record_list = sorted([task for task in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir,task))])
output_EEG_dir = output_dir+"DE_feature/test/"+f"basemean_{basemean}_pt"+"/"
if os.path.isdir(output_EEG_dir)==False:
    os.makedirs(output_EEG_dir)

    
for record in tqdm(record_list):
    file = os.path.join(dataset_dir,record)
    data_inter_E,data_inter_C,label_inter = apply_mixup_test(file, window_size,label_class,basemean)
    trials = data_inter_E.shape[0]

    for trial in range(0,trials):
        output_data_cnn_path = output_EEG_dir+record.split(".")[0]+f"_trial{str(trial+1).zfill(2)}"+"_win_"+str(window_size)+".pt"
        output_data_cnn_tensor = torch.tensor(data_inter_E[trial])

        torch.save(output_data_cnn_tensor, output_data_cnn_path)

### PSD Feature Extraction